# ALL CURRENT LEVELS

In [1]:
%run get_levels.py

# Track: chord

# Track: degree
  0) PATHWAYS_1_4 | L=1, T=0 | bpm=80 | allowed_degrees=[0,1,2,3]
  1) PATHWAYS_5_7 | L=1, T=1 | bpm=80 | allowed_degrees=[4,5,6]
  2) PATHWAYS | L=1, T=2 | bpm=80
  3) NOTE_1_4 | L=2, T=0 | bpm=72 | allowed_degrees=[0,1,2,3]
  4) NOTE_5_7 | L=2, T=1 | bpm=72 | allowed_degrees=[4,5,6]
  5) NOTE | L=2, T=2 | bpm=72
  6) NOTE_TO_TONIC | L=3, T=0 | bpm=72 | allowed_degrees=[1,2,3,4,5,6]
  7) NOTE_AFTER_TONIC | L=3, T=1 | bpm=72 | allowed_degrees=[1,2,3,4,5,6]
  8) NOTE_WITH_TONIC | L=3, T=2 | bpm=72 | allowed_degrees=[1,2,3,4,5,6]
  9) PATHWAYS_2OCT | L=11, T=0 | bpm=120
  10) NOTE_WITH_TONIC_2OCT | L=11, T=1 | bpm=120 | allowed_degrees=[1,2,3,4,5,6]
  11) NOTE_2OCT | L=11, T=2 | bpm=72

# Track: melody
  0) MELODY60_STEP1 | L=111, T=0 | bpm=60
  1) MELODY60_STEP2 | L=111, T=1 | bpm=60
  2) MELODY60 | L=111, T=2 | bpm=60
  3) MELODY90_STEP1 | L=112, T=0 | bpm=90
  4) MELODY90_STEP2 | L=112, T=1 | bpm=60
  5) MELODY90 | L=112, T=2 | bpm=60
  6) MELODY120_STE

In [2]:
# Path setup so Python can find the compiled _earcore module and the Python package
import sys, os
from pathlib import Path

def add_paths():
    cwd = Path.cwd()
    # Walk up to locate eartrainer/eartrainer_Cpp/{build,python}
    for base in [cwd, *cwd.parents]:
        build = base / 'eartrainer' / 'eartrainer_Cpp' / 'build'
        py = base / 'eartrainer' / 'eartrainer_Cpp' / 'python'
        added = False
        if build.exists():
            sys.path.insert(0, str(build))
            added = True
        if py.exists():
            sys.path.insert(0, str(py))
            added = True
        if added:
            return build, py
    return None, None

build_path, py_path = add_paths()
print('Using build path:', build_path)
print('Using python path:', py_path)


Using build path: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/build
Using python path: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/python


In [3]:
# Imports
from eartrainer.session_engine import SessionEngine
from eartrainer.models import SessionSpec, ResultMetrics, ResultReport, SessionSummary, TypedPayload

engine = SessionEngine()
print('Capabilities:', engine.capabilities())

from eartrainer import QuestionBundle as QuestionBundleModel, SimpleMidiPlayer
from eartrainer import _earcore


Capabilities: {'assists': ['Replay', 'GuideTone', 'TempoDown', 'PathwayHint'], 'drills': ['note', 'interval', 'melody', 'chord'], 'session_assists': ['ScaleArpeggio', 'Tonic'], 'version': 'v1'}


In [4]:
import sys
from pathlib import Path

import yaml

PROJECT_ROOT = Path.cwd()
CPP_ROOT = PROJECT_ROOT
CATALOG_PATH = CPP_ROOT / 'resources' / 'degree_levels.yml'
with CATALOG_PATH.open('r', encoding='utf-8') as fh:
    CATALOG_DATA = yaml.safe_load(fh)

if not isinstance(CATALOG_DATA, dict) or 'drills' not in CATALOG_DATA:
    raise ValueError(f'Unexpected catalog format in {CATALOG_PATH}')

def build_catalog_index(data):
    catalog = {}
    for entry in data.get('drills', []):
        if not isinstance(entry, dict):
            continue
        level = int(entry.get('level', -1))
        tier = int(entry.get('tier') or 0)
        catalog.setdefault(level, {}).setdefault(tier, []).append(entry)
    return catalog

catalog_index = build_catalog_index(CATALOG_DATA)
drill_by_id = {
    entry['id']: entry
    for entry in CATALOG_DATA.get('drills', [])
    if isinstance(entry, dict) and 'id' in entry
}

print('Project root:', PROJECT_ROOT)
print('Adaptive catalog:', CATALOG_PATH)
print('Loaded _earcore:', getattr(_earcore, '__file__', '<built-in>'))
print('Catalog levels -> tiers:')
for level in sorted(catalog_index):
    tier_desc = ', '.join(
        f"tier {tier}: {[entry['id'] for entry in entries]}"
        for tier, entries in sorted(catalog_index[level].items())
    )
    print(f'  Level {level}: {tier_desc}')



Project root: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp
Adaptive catalog: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/resources/degree_levels.yml
Loaded _earcore: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/python/eartrainer/_earcore.cpython-311-darwin.so
Catalog levels -> tiers:
  Level 1: tier 0: ['PATHWAYS_1_4'], tier 1: ['PATHWAYS_5_7'], tier 2: ['PATHWAYS']
  Level 2: tier 0: ['NOTE_1_4'], tier 1: ['NOTE_5_7'], tier 2: ['NOTE']
  Level 3: tier 0: ['NOTE_TO_TONIC'], tier 1: ['NOTE_AFTER_TONIC'], tier 2: ['NOTE_WITH_TONIC']
  Level 11: tier 0: ['PATHWAYS_2OCT'], tier 1: ['NOTE_WITH_TONIC_2OCT'], tier 2: ['NOTE_2OCT']


In [5]:
# MIDI playback helper
if "midi_player" in globals():
    try:
        midi_player.stop_all()
    except Exception:
        pass

try:
    midi_player = SimpleMidiPlayer()
    print("SimpleMidiPlayer ready.")
except Exception as exc:
    midi_player = None
    print(f"SimpleMidiPlayer unavailable: {exc}")


fluidsynth: warning: SDL3 not initialized, SDL3 audio driver won't be usable. Have you called SDL_Init(SDL_INIT_AUDIO) ?


SimpleMidiPlayer ready.


In [6]:
import time
from pprint import pprint

LEVEL = 2
TIER = 1
N_QUESTIONS = 5
PAUSE_SECONDS = 1.0
SEED = 124

available_tiers = sorted(catalog_index.get(LEVEL, {}).keys())
print(f"Available tiers for level {LEVEL}: {available_tiers}")

tier_entries = catalog_index.get(LEVEL, {}).get(TIER)
if not tier_entries:
    raise ValueError(f"No drills configured for level {LEVEL}, tier {TIER}")

target_ids = [entry["id"] for entry in tier_entries]

adaptive = _earcore.AdaptiveDrills(str(CATALOG_PATH.parent), seed=SEED)
if hasattr(adaptive, 'set_bout_from_catalog'):
    adaptive.set_bout_from_catalog(LEVEL, CATALOG_DATA)
else:
    adaptive.set_bout(LEVEL)

diagnostic = adaptive.diagnostic()
print("Adaptive diagnostic:")
pprint(diagnostic)

slot_ids = diagnostic.get("ids", [])
id_to_slot = {slot_id: idx for idx, slot_id in enumerate(slot_ids)}

missing = [drill_id for drill_id in target_ids if drill_id not in id_to_slot]
if missing:
    raise RuntimeError(f"AdaptiveDrills missing expected drills for tier: {missing}")

print(f"Target tier drill ids: {target_ids}")

accepted = 0
attempts = 0
MAX_ATTEMPTS = 200

while accepted < N_QUESTIONS and attempts < MAX_ATTEMPTS:
    raw_bundle = adaptive.next()
    attempts += 1

    bundle = QuestionBundleModel.from_json(raw_bundle)
    diag_after = adaptive.diagnostic()
    last_pick_idx = diag_after.get("last_pick")
    picked_id = slot_ids[last_pick_idx] if last_pick_idx is not None else None

    if picked_id not in target_ids:
        continue

    print(f"[{accepted + 1}] {bundle.question_id} ({picked_id}) -> {bundle.question.payload}")
    if midi_player is not None:
        try:
            midi_player.play_prompt(bundle.prompt)
        except Exception as exc:
            print(f"  Playback failed: {exc}")
            midi_player = None
    else:
        print("  Playback skipped (SimpleMidiPlayer unavailable).")

    time.sleep(PAUSE_SECONDS)
    accepted += 1

if accepted < N_QUESTIONS:
    print(f"Stopped after {accepted} questions; adjust SEED or MAX_ATTEMPTS if needed.")
else:
    print("Completed adaptive tier preview.")


Available tiers for level 2: [0, 1, 2]
Adaptive diagnostic:
{'current_track': 'degree',
 'families': ['note', 'note', 'note'],
 'ids': ['NOTE_1_4', 'NOTE_5_7', 'NOTE'],
 'last_pick': None,
 'last_track_pick': 0,
 'level': 2,
 'phase_consistent': False,
 'phase_digit': 0,
 'pick_counts': [0, 0, 0],
 'questions_emitted': 0,
 'resources_dir': '/Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/resources',
 'slots': 3,
 'track_levels': [2, 111, 0],
 'track_weights': [2, 0, 0],
 'tracks': [{'current_level': 2,
             'levels_in_scope': [2, 3],
             'name': 'degree',
             'weight': 2},
            {'current_level': 111,
             'levels_in_scope': [],
             'name': 'melody',
             'weight': 0},
            {'current_level': 0,
             'levels_in_scope': [],
             'name': 'chord',
             'weight': 0}]}
Target tier drill ids: ['NOTE_5_7']
[1] ad-001 (NOTE_5_7) -> {'degree': 4, 'midi': 67, 'tonic_midi': 60}
[2] ad-003 (NO

In [7]:
import time
PLAYMIDI = True
LEVELS = [11, 111, 0]
adaptive = _earcore.AdaptiveDrills(str(CATALOG_PATH.parent), seed=SEED)
adaptive.set_bout(LEVELS)
diagnostic = adaptive.diagnostic()

slot_ids = diagnostic.get("ids", [])

for i in range(10):
    raw_bundle = adaptive.next()
    bundle = QuestionBundleModel.from_json(raw_bundle)
    
    diag_after = adaptive.diagnostic()
    last_pick_idx = diag_after.get("last_pick")
    
    picked_id = slot_ids[last_pick_idx] if last_pick_idx is not None else None

    print(f"[{i + 1}] {bundle.question_id} ({picked_id}) -> {bundle.question.payload}")
    if PLAYMIDI:
        if midi_player is not None:
            try:
                midi_player.play_prompt(bundle.prompt)
            except Exception as exc:
                print(f"  Playback failed: {exc}")
                midi_player = None
        else:
            print("  Playback skipped (SimpleMidiPlayer unavailable).")

        time.sleep(1)

adaptive.diagnostic()



[1] ad-001 (NOTE_WITH_TONIC_2OCT) -> {'degree': 4, 'midi': 55, 'tonic_midi': 60}
[2] ad-002 (NOTE_2OCT) -> {'degree': 1, 'midi': 50, 'tonic_midi': 60}
[3] ad-003 (NOTE_WITH_TONIC_2OCT) -> {'degree': 6, 'midi': 71, 'tonic_midi': 60}
[4] ad-004 (NOTE_WITH_TONIC_2OCT) -> {'degree': 1, 'midi': 50, 'tonic_midi': 60}
[5] ad-005 (PATHWAYS_2OCT) -> {'degree': 2, 'midi': 64, 'tonic_midi': 60}
[6] ad-006 (PATHWAYS_2OCT) -> {'degree': 5, 'midi': 57, 'tonic_midi': 60}
[7] ad-007 (NOTE_2OCT) -> {'degree': 5, 'midi': 69, 'tonic_midi': 60}
[8] ad-008 (PATHWAYS_2OCT) -> {'degree': 4, 'midi': 67, 'tonic_midi': 60}
[9] ad-009 (NOTE_WITH_TONIC_2OCT) -> {'degree': 3, 'midi': 53, 'tonic_midi': 60}
[10] ad-010 (NOTE_WITH_TONIC_2OCT) -> {'degree': 1, 'midi': 62, 'tonic_midi': 60}


{'current_track': 'degree',
 'families': ['note', 'note', 'note'],
 'ids': ['PATHWAYS_2OCT', 'NOTE_WITH_TONIC_2OCT', 'NOTE_2OCT'],
 'last_pick': 1,
 'last_track_pick': 0,
 'level': 11,
 'phase_consistent': True,
 'phase_digit': 1,
 'pick_counts': [3, 5, 2],
 'questions_emitted': 10,
 'resources_dir': '/Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/resources',
 'slots': 3,
 'track_levels': [11, 111, 0],
 'track_weights': [1, 3, 0],
 'tracks': [{'current_level': 11,
   'levels_in_scope': [11],
   'name': 'degree',
   'weight': 1},
  {'current_level': 111,
   'levels_in_scope': [111, 112, 113],
   'name': 'melody',
   'weight': 3},
  {'current_level': 0, 'levels_in_scope': [], 'name': 'chord', 'weight': 0}]}